In [1]:
!pip install scikit-optimize
!pip install tqdm_joblib
# --- Standard Library Imports ---
import os
import random
import re
import warnings
from pathlib import Path

# --- Google Colab / IPython Imports ---
from google.colab import drive
from IPython.display import Image

# --- Third-Party Library Imports ---
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import torch
import multiprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from joblib import Parallel, delayed
from scipy.stats import skew, kurtosis
from skimage import exposure
from skimage.color import rgb2gray
from skimage.feature import hog, local_binary_pattern
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real
from transformers import ViTModel, ViTImageProcessor, ViTFeatureExtractor
from tqdm_joblib import tqdm_joblib
from tqdm.auto import tqdm

# --- Scikit-Learn Imports ---
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import graycomatrix, graycoprops
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree

# --- XGBoost Imports ---
from xgboost import XGBClassifier

# --- Configuration ---
random.seed(42)
warnings.filterwarnings("ignore")


In [2]:
base_dir = Path('/content/skin-cancer-mnist-ham10000')
zip_file = Path('skin-cancer-mnist-ham10000.zip')

# Download zip file from Drive if file doesn't exist
if not zip_file.exists():
    !gdown https://drive.google.com/file/d/1wS-ExXzOFyF8evYLi6kNhhFkF0-uFBxq/view?usp=sharing --fuzzy
else:
    print("ZIP file already exists, skipping download.")

# Unzip if data folder doesn't exist
if not base_dir.exists():
    !unzip skin-cancer-mnist-ham10000.zip
else:
    print("Data directory already exists, skipping extraction.")

ZIP file already exists, skipping download.
Data directory already exists, skipping extraction.


# EDA (TODO)

## Visualization & EDA Helper Functions

In [3]:
def whoAmI(img):
    """
    Prints dtype, min, max, mean, height, and width of an image.
    """
    print(f"dtype     : {img.dtype}")
    print(f"min       : {np.min(img):.4f}")
    print(f"max       : {np.max(img):.4f}")
    print(f"mean      : {np.mean(img):.4f}")
    print(f"shape     : {img.shape}")
    if img.ndim == 2:
        print(f"height    : {img.shape[0]}")
        print(f"width     : {img.shape[1]}")
    elif img.ndim == 3:
        print(f"height    : {img.shape[0]}")
        print(f"width     : {img.shape[1]}")
        print(f"channels  : {img.shape[2]}")

#VS - II helpful functions

# summarizes a data frame in a printer friendly way
def summarize_dataframe(df):
    summary = pd.DataFrame({
        'Column Name': df.columns,
        'Data Type': df.dtypes.values,
        'Null Count': df.isnull().sum().values,
        'Non-Null Count': df.notnull().sum().values,
        'Unique Count': df.nunique().values
    })
    return summary

def plot_training_history(history):
    """
    Plots training and validation loss & accuracy from a Keras history object.

    Parameters:
    - history: Keras history object from model.fit()
    """

    epochs = range(1, len(history.history['loss']) + 1)

    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']


    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].plot(epochs, train_loss, label='Training Loss', color='blue', linestyle='-')
    ax[0].plot(epochs, val_loss, label='Validation Loss', color='red', linestyle='--')
    ax[0].set_title('Training & Validation Loss')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].legend()

    ax[1].plot(epochs, train_acc, label='Training Accuracy', color='blue', linestyle='-')
    ax[1].plot(epochs, val_acc, label='Validation Accuracy', color='red', linestyle='--')
    ax[1].set_title('Training & Validation Accuracy')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Accuracy')
    ax[1].legend()

    plt.tight_layout()
    plt.show()



# Preprocessing Helper Functions

In [4]:
# ---------------------- Hair Removal ----------------------
def remove_hairs(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, thresh = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    result = cv2.inpaint(img, thresh, inpaintRadius=1, flags=cv2.INPAINT_TELEA)
    return result


# ---------------------- Scale Marker Masking ----------------------
def mask_scale_markers(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h > 500 and (w/h > 5 or h/w > 5):
            img[y:y+h, x:x+w] = 0
    return img


# ---------------------- Combined Preprocessing ----------------------
def preprocess_image(img):
    img_no_hair = remove_hairs(img)
    img_clean = mask_scale_markers(img_no_hair)
    return img_clean

# ---------------------- Worker Parallelization Function  ----------------------
def process_single_image_entry(image_path, label, image_id, resize_shape):
    """
    Worker function to load, preprocess, and resize a single image.
    Returns a tuple of (image, label, image_id).
    """
    if os.path.exists(image_path):
        # Load and convert image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Apply your existing preprocessing steps
        img = preprocess_image(img) # This calls remove_hairs and mask_scale_markers

        # Resize
        img = cv2.resize(img, resize_shape)
        return img, label, image_id

    # Return None if the file path is invalid
    return None, None, None

# ---------------------- Master Parallelization Function  ----------------------
def preprocess_all_images_parallel(df, resize_shape=(224, 224)):
    """
    Preprocesses all images from the DataFrame in parallel with a progress bar.
    """
    # Use the tqdm_joblib context manager for a clean progress bar
    with tqdm_joblib(desc="Preprocessing Images", total=len(df)) as progress_bar:
        results = Parallel(n_jobs=-1, verbose=0)(
            delayed(process_single_image_entry)(
                row['path'], row['dx'], row['image_id'], resize_shape
            )
            for _, row in df.iterrows()
        )

    # Filter out any failed results (where the image path was invalid)
    valid_results = [res for res in results if res[0] is not None]

    # Unzip the list of tuples into separate lists
    images, labels, image_ids = zip(*valid_results)

    return np.array(images), list(labels), list(image_ids)

# Feature Extraction Helper Functions

## Simple Feature Extraction

In [5]:
# HSV Features
def compute_hsv_histogram(img, h_bins=16, s_bins=8, v_bins=8):
    """
    Compute a normalized HSV color histogram from an RGB image.

    Args:
        img (np.ndarray): RGB image.
        h_bins, s_bins, v_bins: Number of bins for H, S, and V channels.

    Returns:
        np.ndarray: Flattened and normalized HSV histogram.
    """
    # Convert from RGB to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # Compute 3D histogram
    hist = cv2.calcHist(
        [hsv], [0, 1, 2], None,
        [h_bins, s_bins, v_bins],
        [0, 180, 0, 256, 0, 256]
    )

    # Normalize and flatten
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_hsv_features_from_array(image_array, bins=(16, 8, 8)):
    h_bins, s_bins, v_bins = bins
    with tqdm_joblib(desc="Extracting HSV Features", total=len(image_array)) as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_hsv_histogram)(img, h_bins, s_bins, v_bins) for img in image_array
        )
    return np.array(features)

In [6]:
# HoG Features
def compute_hog(img):
    gray_img = rgb2gray(img)
    features = hog(
        gray_img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        visualize=False
    )
    return features

def extract_hog_features_from_array(image_array):
    with tqdm_joblib(desc="Extracting HOG Features", total=len(image_array)) as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_hog)(img) for img in image_array
        )
    return np.array(features)

In [7]:
# LBP Features
def compute_lbp_features(img, radius=1, n_points=8, method='uniform'):
    """
    Compute a normalized LBP histogram from a grayscale version of the RGB image.

    Args:
        img (np.ndarray): RGB image.
        radius (int): Radius of circle.
        n_points (int): Number of sampling points.
        method (str): LBP method ('uniform', 'default', etc.).

    Returns:
        np.ndarray: Normalized LBP histogram (1D feature vector).
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lbp = local_binary_pattern(gray, n_points, radius, method=method)
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist




def extract_lbp_features_from_array(image_array, radius=1, n_points=8, method='uniform'):
    with tqdm_joblib(desc="Extracting LBP Features", total=len(image_array)) as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_lbp_features)(img, radius, n_points, method) for img in image_array
        )
    return np.array(features)

In [8]:
# Color Features
def compute_color_features(img, hist_bins=(8, 8, 8)):
    """
    Computes color statistics (mean, std) and a normalized color histogram.

    Args:
        img (np.ndarray): RGB image.
        hist_bins (tuple): Number of bins for R, G, and B channels.

    Returns:
        np.ndarray: A concatenated 1D feature vector containing
                    mean RGB, std dev RGB, and the flattened normalized histogram.
    """
    # 1. Color Statistics
    mean_rgb = img.mean(axis=(0, 1))
    std_rgb = img.std(axis=(0, 1))

    # 2. Color Histogram
    hist = cv2.calcHist(
        [img], [0, 1, 2], None, hist_bins,
        [0, 256, 0, 256, 0, 256]
    )

    # Normalize and flatten
    hist = cv2.normalize(hist, hist).flatten()

    return np.concatenate([mean_rgb, std_rgb, hist])

def extract_color_features_from_array(image_array, hist_bins=(8, 8, 8)):
    """Extracts color features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting Color Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_color_features)(img, hist_bins) for img in image_array
        )
    return np.array(features)


In [9]:
# Statistical features
def compute_intensity_stats(img):
    """
    Computes statistical moments of the pixel intensities.

    Args:
        img (np.ndarray): RGB image.

    Returns:
        np.ndarray: A 1D feature vector containing mean, standard deviation,
                    skewness, and kurtosis of the grayscale image.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    pixels = gray.flatten()

    features = [
        np.mean(pixels),
        np.std(pixels),
        skew(pixels),
        kurtosis(pixels)
    ]
    return np.array(features)


def extract_intensity_stats_from_array(image_array):
    """Extracts intensity statistics from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting Intensity Stats") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_intensity_stats)(img) for img in image_array
        )
    return np.array(features)


In [10]:
# GLCM Features
def compute_glcm_features(img, distances=[5], angles=[0]):
    """
    Computes texture features using a Gray-Level Co-occurrence Matrix (GLCM).

    Args:
        img (np.ndarray): RGB image.
        distances (list): List of pixel pair distance offsets.
        angles (list): List of pixel pair angles in radians.

    Returns:
        np.ndarray: A 1D feature vector containing contrast, dissimilarity,
                    homogeneity, ASM, energy, and correlation.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    glcm = graycomatrix(
        gray, distances=distances, angles=angles,
        levels=256, symmetric=True, normed=True
    )

    props = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']
    features = [graycoprops(glcm, prop)[0, 0] for prop in props]

    return np.array(features)

def extract_glcm_features_from_array(image_array, distances=[5], angles=[0]):
    """Extracts GLCM features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting GLCM Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_glcm_features)(img, distances, angles) for img in image_array
        )
    return np.array(features)


In [11]:
# Gabor Features
def compute_gabor_features(img, resize_dim=(224, 224), orientations=6, scales=(1, 3, 5, 7)):
    """
    Computes features using a bank of Gabor filters.

    Args:
        img (np.ndarray): RGB image.
        resize_dim (tuple): Dimensions to resize the image to. Set to None to skip.
        orientations (int): Number of filter orientations.
        scales (tuple): A tuple of sigma values for the Gabor kernels.

    Returns:
        np.ndarray: A 1D vector containing the mean and std dev of each filtered response.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    if resize_dim:
        gray = cv2.resize(gray, resize_dim)

    features = []
    thetas = np.linspace(0, np.pi, orientations, endpoint=False)

    for theta in thetas:
        for sigma in scales:
            kernel = cv2.getGaborKernel(
                ksize=(21, 21), sigma=sigma, theta=theta,
                lambd=10.0, gamma=0.5, psi=0, ktype=cv2.CV_32F
            )
            filtered = cv2.filter2D(gray, ddepth=-1, kernel=kernel)
            features.extend([filtered.mean(), filtered.std()])

    return np.array(features)


def extract_gabor_features_from_array(image_array, resize_dim=(224, 224), orientations=6, scales=(1, 3, 5, 7)):
    """Extracts Gabor features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting Gabor Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_gabor_features)(img, resize_dim, orientations, scales) for img in image_array
        )
    return np.array(features)

In [12]:
# Shape Features
def compute_shape_features(img):
    """
    Computes shape features from the largest contour in the image.

    Args:
        img (np.ndarray): RGB image.

    Returns:
        np.ndarray: A 1D vector of [area, perimeter, compactness].
                    Returns zeros if no contours are found.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return np.zeros(3)

    cnt = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    if area == 0:
        return np.zeros(3)

    compactness = (perimeter ** 2) / (4 * np.pi * area)

    return np.array([area, perimeter, compactness])

def extract_shape_features_from_array(image_array):
    """Extracts shape features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting Shape Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_shape_features)(img) for img in image_array
        )
    return np.array(features)

In [13]:
# Corner Features
def compute_corner_features(img, max_corners=100, quality_level=0.01, min_distance=10):
    """
    Computes the number of corners detected by the Shi-Tomasi method.

    Args:
        img (np.ndarray): RGB image.
        max_corners (int): Maximum number of corners to return.
        quality_level (float): Parameter for corner detection (0-1).
        min_distance (int): Minimum Euclidean distance between corners.

    Returns:
        np.ndarray: A 1D array with a single feature: the corner count.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    corners = cv2.goodFeaturesToTrack(
        gray, maxCorners=max_corners,
        qualityLevel=quality_level,
        minDistance=min_distance
    )

    num_corners = 0 if corners is None else len(corners)
    return np.array([num_corners])

def extract_corner_features_from_array(image_array, max_corners=100, quality_level=0.01, min_distance=10):
    """Extracts corner features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting Corner Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_corner_features)(img, max_corners, quality_level, min_distance) for img in image_array
        )
    return np.array(features)

In [14]:
# SIFT Features
def compute_sift_features(img, max_features=50):
    """
    Computes SIFT features and returns a fixed-size vector by padding/truncating.

    Args:
        img (np.ndarray): RGB image.
        max_features (int): The target number of descriptors.

    Returns:
        np.ndarray: A flattened 1D feature vector of size (max_features * 128).
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(gray, None)

    if descriptors is None:
        return np.zeros(max_features * 128)

    num_descriptors = descriptors.shape[0]
    if num_descriptors < max_features:
        pad_width = max_features - num_descriptors
        # Pad with zeros to reach max_features
        feature_vec = np.pad(descriptors, ((0, pad_width), (0, 0)), 'constant')
    else:
        # Truncate to keep only the first max_features
        feature_vec = descriptors[:max_features, :]

    return feature_vec.flatten()







def extract_sift_features_from_array(image_array, max_features=50):
    """Extracts fixed-size SIFT features from an array of images in parallel."""
    with tqdm_joblib(total=len(image_array), desc="Extracting SIFT Features") as progress_bar:
        features = Parallel(n_jobs=-1, verbose=0)(
            delayed(compute_sift_features)(img, max_features) for img in image_array
        )
    return np.array(features)

## Complex Feature Extraction

In [15]:
def extract_vit_features_from_array(image_array, model, feature_processor, batch_size=32):
    """
    Extracts ViT [CLS] token embeddings from an array of RGB images using batch processing.

    Args:
        image_array (np.ndarray or list): A list or array of RGB images (H, W, C).
        model (torch.nn.Module): The pre-trained Vision Transformer model.
        feature_processor (ViTImageProcessor): The processor to prepare images for the model.
        batch_size (int): The number of images to process in a single batch.

    Returns:
        np.ndarray: A 2D array where each row is the [CLS] embedding for an image.
    """
    # Determine the device and move the model to it
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # Set the model to evaluation mode

    all_features = []
    num_images = len(image_array)

    # Process images in batches
    with tqdm_joblib(total=num_images, desc="Extracting ViT Features (Batching)") as pbar:
        for i in range(0, num_images, batch_size):
            # Create a batch of images
            batch_images = image_array[i:i + batch_size]

            # The feature processor expects a list of images
            inputs = feature_processor(images=list(batch_images), return_tensors="pt").to(device)

            # Perform inference without calculating gradients
            with torch.no_grad():
                outputs = model(**inputs)

            # Extract the [CLS] token's embeddings
            # The [CLS] token is always the first one
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_features.append(cls_embeddings)

            pbar.update(len(batch_images))

    # Concatenate all batch results into a single NumPy array
    return np.vstack(all_features)


# Model Selection & Configuration

In [28]:
MODELS = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'model__n_estimators': Integer(50, 500),
            'model__max_depth': Integer(5, 50),
            'model__min_samples_split': Integer(2, 12),
            'model__min_samples_leaf': Integer(1, 10),
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
        'params': {
            'model__n_estimators': Integer(50, 500),
            'model__max_depth': Integer(3, 15),
            'model__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'model__subsample': Real(0.6, 1.0),
            'model__colsample_bytree': Real(0.6, 1.0),
        }
    },
    'SVC': {
        'model': SVC(random_state=42, probability=True),
        'params': {
            'model__C': Real(1e-3, 1e3, prior='log-uniform'),
            'model__gamma': Real(1e-4, 1e-1, prior='log-uniform'),
            'model__kernel': Categorical(['rbf', 'poly']),
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'model__n_neighbors': Integer(3, 30),
            'model__weights': Categorical(['uniform', 'distance']),
            'model__p': Integer(1, 2) # 1 for Manhattan, 2 for Euclidean
        }
    },
    'MLPClassifier': {
        'model': MLPClassifier(random_state=42, max_iter=500, early_stopping=True),
        'params': {
            'model__hidden_layer_sizes': Categorical([(50, 50), (100,), (100, 50, 25)]),
            'model__activation': Categorical(['relu', 'tanh']),
            'model__solver': Categorical(['adam']),
            'model__alpha': Real(1e-5, 1e-2, prior='log-uniform'),
            'model__learning_rate': Categorical(['constant', 'adaptive']),
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000, solver='liblinear'),
        'params': {
            'model__penalty': Categorical(['l1', 'l2']),
            'model__C': Real(1e-3, 1e3, prior='log-uniform'),
        }
    }
}

# Hyperparameter Search & Model Evaluation

In [32]:
def run_model_evaluation_pipeline(X, y, n_splits=5, random_state=42):
    """
    Runs a simplified and efficient training and evaluation pipeline.

    - Stratified train-test split.
    - Uses a pipeline to couple the oversampler and the model.
    - Performs Bayesian hyperparameter search with cross-validation on the training set.
    - Evaluates the best model on the held-out test set.

    Args:
        X (np.ndarray): Feature matrix.
        y (np.ndarray): Target labels.
        n_splits (int): Number of folds for cross-validation.
        random_state (int): Seed for reproducibility.

    Returns:
        dict: A dictionary containing the best model, test report, and CV score for each model type.
    """
    # Step 1: Stratified Train-Test Split to create a final hold-out set
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=random_state
    )

    final_results = {}

    # Loop through each model defined in the MODELS dictionary
    for model_name, config in tqdm(MODELS.items(), desc="Evaluating Models"):
        print(f"--- Starting Evaluation for: {model_name} ---")

        # Step 2: Create a pipeline that first oversamples, then fits the model
        # This ensures oversampling only happens on the training data within each CV fold
        pipeline = ImbPipeline([
            ('sampler', RandomOverSampler(sampling_strategy='not majority', random_state=random_state)),
            ('model', config['model'])
        ])

        # Step 3: Set up cross-validation and hyperparameter search
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

        bayes_search = BayesSearchCV(
            estimator=pipeline,
            search_spaces=config['params'],
            n_iter=10,  # Number of parameter settings that are sampled
            cv=cv,
            scoring='f1_macro',
            n_jobs=-1,
            random_state=random_state,
            verbose=1
        )

        # Step 4: Run the search on the training data
        print("Running Bayesian search for best hyperparameters...")
        bayes_search.fit(X_train, y_train)

        # Step 5: Evaluate the best found model on the held-out test set
        print("Evaluating best model on the hold-out test set...")
        best_model = bayes_search.best_estimator_
        y_test_pred = best_model.predict(X_test)
        report_dict = classification_report(y_test, y_test_pred, output_dict=True)

        # Store results
        final_results[model_name] = {
            'best_model': best_model,
            'test_set_report': pd.DataFrame(report_dict).transpose(),
            'best_cv_f1_macro': bayes_search.best_score_,
            'best_params': bayes_search.best_params_
        }

        print(f"Best cross-validation F1-macro score: {bayes_search.best_score_:.4f}")
        print(f"Test Set F1-macro score: {report_dict['macro avg']['f1-score']:.4f}\n")

    return final_results

# Pipeline Execution

## Load Images

In [ ]:
base_dir = Path('/content/skin-cancer-mnist-ham10000/')
file_path = base_dir / 'HAM10000_metadata.csv'
df = pd.read_csv(file_path)

# --- Image Path Gathering ---
image_paths = {}
image_folders = ['HAM10000_images_part_1', 'HAM10000_images_part_2']
for folder in image_folders:
    folder_path = base_dir / folder
    for image_file in folder_path.glob('**/*.jpg'):
        image_id = image_file.stem
        image_paths[image_id] = str(image_file) # Store path as string for cv2

df['path'] = df['image_id'].map(image_paths.get)
df.dropna(inplace=True)

## Preprocess & Extract Features

In [20]:
# Preprocess images
processed_images, raw_labels, image_ids = preprocess_all_images_parallel(df)

Preprocessing Images:   0%|          | 0/9958 [00:00<?, ?it/s]

In [21]:
# Save data before feature extraction (same split as in our k-folds cross validation)
random_state = 42
X_train_paths, X_test_paths, y_train_paths, y_test_paths = train_test_split(
    df['path'], df['dx'], test_size=0.15, stratify=df['dx'], random_state=random_state
)
np.savez('final_unbalanced_data.npz', X_train=X_train_paths, y_train=y_train_paths, X_test=X_test_paths, y_test=y_test_paths)

In [22]:
# Extract Simple Features
X_hog = extract_hog_features_from_array(processed_images)
X_hsv = extract_hsv_features_from_array(processed_images)
X_lbp = extract_lbp_features_from_array(processed_images)
X_color = extract_color_features_from_array(processed_images)
X_intensity = extract_intensity_stats_from_array(processed_images)
X_glcm = extract_glcm_features_from_array(processed_images)
X_gabor = extract_gabor_features_from_array(processed_images)
X_shape = extract_shape_features_from_array(processed_images)
X_corners = extract_corner_features_from_array(processed_images)
X_sift = extract_sift_features_from_array(processed_images)

Extracting HOG Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting HSV Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting LBP Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting Color Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting Intensity Stats:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting GLCM Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting Gabor Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting Shape Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting Corner Features:   0%|          | 0/9958 [00:00<?, ?it/s]

Extracting SIFT Features:   0%|          | 0/9958 [00:00<?, ?it/s]

In [23]:
# Extract Complex Features
MODEL_NAME = 'google/vit-base-patch16-224-in21k'
vit_model = ViTModel.from_pretrained(MODEL_NAME) # Load in ViT model
vit_processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

X_vit = extract_vit_features_from_array(
    image_array=processed_images,
    model=vit_model,
    feature_processor=vit_processor,
    batch_size=32
)

Extracting ViT Features (Batching):   0%|          | 0/9958 [00:00<?, ?it/s]

In [25]:
# Create a list of features
X_feature_sets = {
    "hog": X_hog,
    "hsv": X_hsv,
    "lbp": X_lbp,
    "color": X_color,
    "intensity": X_intensity,
    "glcm": X_glcm,
    "gabor": X_gabor,
    "shape": X_shape,
    "corners": X_corners,
    "sift": X_sift,
    "ViT": X_vit
}

In [26]:
# Encode Labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(raw_labels)

In [ ]:
# Save the Features
feature_filename = "final_features.npz"
np.savez_compressed(
    feature_filename,
    **X_feature_sets,
    labels=y,
    label_map=label_encoder.classes_
)

## **Feature Visualization (PCA & tSNE) TODO**

## Execute Pipeline

In [ ]:
feature_set_leaderboard = {}
for feature_name, X_features in tqdm(X_feature_sets.items(), desc="Evaluating Feature Sets"):
    print(f"\n{'='*20} Evaluating Feature Set: {feature_name.upper()} {'='*20}")

    # 1. Run the model evaluation pipeline for the current feature set
    # This will test all models (RandomForest, XGBoost, etc.) on this feature set
    final_results = run_model_evaluation_pipeline(X_features, y)

    # 2. Determine the best model from the results for this feature set
    # We find the model with the highest cross-validation F1-macro score
    best_model_name = max(
        final_results,
        key=lambda model: final_results[model]['best_cv_f1_macro']
    )
    best_model_score = final_results[best_model_name]['best_cv_f1_macro']

    # 3. Store the winner for this feature set
    feature_set_leaderboard[feature_name] = {
        'best_model': best_model_name,
        'cv_f1_macro': best_model_score
    }

    # 4. Print a summary for the current feature set
    print(f"\n--- Results for Feature Set: {feature_name.upper()} ---")
    # Create a summary DataFrame for easy viewing
    summary_data = {
        model: res['best_cv_f1_macro']
        for model, res in final_results.items()
    }
    df_summary = pd.DataFrame.from_dict(
        summary_data, orient='index', columns=['best_cv_f1_macro']
    )
    df_summary = df_summary.sort_values(by='best_cv_f1_macro', ascending=False)
    print(df_summary)
    print(f"\nBest performing model for '{feature_name}': {best_model_name} (CV F1 Score: {best_model_score:.4f})")

print(f"\n{'='*25} FINAL LEADERBOARD {'='*25}")
df_leaderboard = pd.DataFrame(feature_set_leaderboard).T
df_leaderboard = df_leaderboard.sort_values(by='cv_f1_macro', ascending=False)
print(df_leaderboard)

Evaluating Feature Sets:   0%|          | 0/11 [00:00<?, ?it/s]


==================== Evaluating Feature Set: HOG ====================


Evaluating Models:   0%|          | 0/6 [00:00<?, ?it/s]

--- Starting Evaluation for: RandomForest ---
Running Bayesian search for best hyperparameters...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
